In [13]:
import pandas as pd
from datetime import timedelta
from types import new_class
from dash import  dcc
from dash import  html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from django_plotly_dash import DjangoDash
import pandas as pd
from datetime import timedelta

import plotly.express as px


In [3]:
df1 =pd.read_excel('Retail_kaggle.xlsx',parse_dates=['InvoiceDate'])
df=df1.copy()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   lower        1816 non-null    object        
 3   Description  540455 non-null  object        
 4   Quantity     541909 non-null  int64         
 5   InvoiceDate  541909 non-null  datetime64[ns]
 6   UnitPrice    541909 non-null  float64       
 7   CustomerID   406829 non-null  float64       
 8   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 37.2+ MB


In [5]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   lower        1816 non-null    object        
 3   Description  540455 non-null  object        
 4   Quantity     541909 non-null  int64         
 5   InvoiceDate  541909 non-null  datetime64[ns]
 6   UnitPrice    541909 non-null  float64       
 7   CustomerID   406829 non-null  float64       
 8   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 37.2+ MB


In [9]:
df['Total'] = df['Quantity'] * df['UnitPrice']
day_date = df['InvoiceDate'].max() + timedelta(days=1)


In [10]:
day_date

Timestamp('2011-12-10 12:50:00')

In [11]:
df_process = df.groupby(['CustomerID']).agg({
                'InvoiceDate': lambda x: (day_date - x.max()).days,
                'InvoiceNo': 'count',
                'Total': 'sum'})
df_process.rename(columns={'InvoiceDate': 'Recency',
                                'InvoiceNo': 'Frequency',
                                'Total': 'MonetaryValue'}, inplace=True)

In [12]:
df_process

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40
...,...,...,...
18280.0,278,10,180.60
18281.0,181,7,80.82
18282.0,8,13,176.60


In [15]:
def label_rfm_segments(df):
    if df['score'] >= 9:
        return 'Cant Loose Them'
    elif ((df['score'] >= 8) and (df['score'] < 9)):
        return 'Champions'
    elif ((df['score'] >= 7) and (df['score'] < 8)):
        return 'Loyal'
    elif ((df['score'] >= 6) and (df['score'] < 7)):
        return 'Potential'
    elif ((df['score'] >= 5) and (df['score'] < 6)):
        return 'Promising'
    elif ((df['score'] >= 4) and (df['score'] < 5)):
        return 'Needs Attention'
    else:
        return 'Others'
def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])#, encoding = "iso-8859-1",engine="openpyxl"


In [17]:
r_labels = range(4, 0, -1); f_labels = range(1, 5);m_labels = range(1, 5)
r_groups = pd.qcut(df_process['Recency'], q=4, labels=r_labels)# Assign these labels to 4 equal percentile groups 
f_groups = pd.qcut(df_process['Frequency'], q=4, labels=f_labels)# Create new columns R and F 
df_process = df_process.assign(R = r_groups.values, F = f_groups.values)
m_groups = pd.qcut(df_process['MonetaryValue'], q=4, labels=m_labels)# Create new column M
df_process = df_process.assign(M = m_groups.values)
df_process['rfm_Concat'] = df_process.apply(join_rfm, axis=1)

rfm_count_unique = df_process.groupby('rfm_Concat')['rfm_Concat'].nunique()
df_process['score'] = df_process[['R','F','M']].sum(axis=1)

In [18]:
df_process['RFM_Level'] = df_process.apply(label_rfm_segments, axis=1)
df_process.reset_index(inplace=True)
rfm_level_agg = df_process.groupby('RFM_Level').agg({'Recency': 'mean','Frequency': 'mean','MonetaryValue': 'mean'}).round(1)

rfm_level_agg.reset_index(inplace = True)


In [19]:
rfm_count_unique

rfm_Concat
111    1
112    1
113    1
114    1
121    1
      ..
434    1
441    1
442    1
443    1
444    1
Name: rfm_Concat, Length: 62, dtype: int64

In [20]:
rfm_level_agg

,RFM_Level,Recency,Frequency,MonetaryValue
0,Cant Loose Them,25.2,195.1,4130.3
1,Champions,62.7,57.0,974.7
2,Loyal,78.8,39.7,724.2
3,Needs Attention,174.5,13.9,227.1
4,Others,264.8,7.8,109.1
5,Potential,94.3,28.5,491.8
6,Promising,153.0,21.2,346.8


In [21]:
df_mod = df_process[['CustomerID', 'score', 'RFM_Level','Recency','Frequency','MonetaryValue']]
df_norm = df[['CustomerID', 'Country']]
df_norm.drop_duplicates(subset = ['CustomerID'], inplace=True)
df_final = pd.merge(df_mod, df_norm, on='CustomerID', how='left')
df_graph = df_final[['Country','RFM_Level']].copy()
df_graph['RFM_Level_cnt'] = df_graph.groupby(['Country','RFM_Level'])['Country'].transform('count')
df_graph.drop_duplicates(['Country', 'RFM_Level','RFM_Level_cnt'], inplace=True)
df_graph.sort_values(by = ['Country'], inplace=True)
df_graph.reset_index()

<ipython-input-21-0273bba52f0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_norm.drop_duplicates(subset = ['CustomerID'], inplace=True)


,index,Country,RFM_Level,RFM_Level_cnt
0,34,Australia,Cant Loose Them,3
1,60,Australia,Loyal,2
2,37,Australia,Champions,2
3,33,Australia,Needs Attention,2
4,11,Austria,Cant Loose Them,3
...,...,...,...,...
125,410,United Kingdom,Needs Attention,369
126,0,United Kingdom,Others,361
127,2935,Unspecified,Loyal,1
128,327,Unspecified,Champions,1
